In [1]:
import os 
import pandas as pd
import numpy as np
from ast import literal_eval
from collections import Counter
from operator import itemgetter

### File/dir locations

In [3]:
DATA_DIR = os.getenv("DATA_DIR")
filename = "preprocessed_taxon_pageseq_20190114_20190116.csv.gz"
df_file = os.path.join(DATA_DIR, "processed_journey", filename)
df_rel_file = os.path.join(DATA_DIR, "processed_journey", "rel_"+filename)

### Load up unfiltered data: 14-16/01/19

In [ ]:
df = pd.read_csv(df_file, sep="\t", compression = "gzip")
df.shape

In [ ]:
df.columns

Original dataset shape (5048130, 17)

In [ ]:
df.drop(['Dates','Page_List', 'Event_List', 'Taxon_List', 'Taxon_Page_List', 'Page_List_NL'],axis=1,inplace=True)

In [ ]:
df.to_csv("reduced_"+filename, compression="gzip", index=False)

In [ ]:
## Volume of traffic
df.Occurrences.sum()

In [ ]:
def column_eval(cols):
    for column in cols:
    print(column)
    if not isinstance(df[column].iloc[0],list):
        df[column] = df[column].map(literal_eval)

In [ ]:
columns = ['Page_Event_List', 'DeviceCategories', 'Event_cats_agg', 'Event_cat_act_agg',]
column_eval(columns)

Run this at some point to compare against "related content" link-specific dataset. 
There seems to be an indication that there are no major differences

In [ ]:
# og_df_devices = Counter()
# for item in df.DeviceCategories.values:
#     for key,value in item:
#         og_df_devices[key]+=value

In [ ]:
def more_device(x,device):
    return max(x,key=itemgetter(1))[0]==device 
def device_count(x,device):
    return sum([value for item, value in x if item==device])      

In [ ]:
df["DesktopCount"] = df['DeviceCategories'].map(lambda x: device_count(x,"desktop"))
df["MobileCount"] = df['DeviceCategories'].map(lambda x: device_count(x,"mobile"))        

### Out of all journeys, what is the % (volume, sum of Occurrences) of journeys including "related content" link clicks?
From previous analysis, I want to say roughly 10%, but let's verify that

In [ ]:
# df_relc = df_rel[df_rel.Sequence.str.contains("Related content")].copy(deep=True)
## Sum of occurrences of journeys including "related content" link clicks (excluding "explore this topic ones")

In [ ]:
(444000*100)/df.Occurrences.sum()

### Out of all journeys coming from mobiles/desktops, what is the % (volume, sum of Occurrences) of journeys including "related content" link clicks?

In [ ]:
df["MoreDesktops"] = df['DeviceCategories'].map(lambda x: more_device(x,"desktop"))